In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import metrics
file2 = "loan_data_set.csv.xls"
test_df = pd.read_csv(file2)

In [3]:
# our SME pointed out that these features don't have an impact on Loan_Status:
# ["Gender","Married","Dependents","Education",]
import numpy as np
#test1 77%
# features = test_df.drop(columns=["Loan_ID","Loan_Status"])
#test2 80%
features = test_df.drop(columns=["Loan_ID","Loan_Status","Loan_Amount_Term","Property_Area"])
#test3 76%
# features = test_df.drop(columns=["Loan_ID","Loan_Status","Loan_Amount_Term","Property_Area","Self_Employed"])
features = pd.get_dummies(features)
X = features
y = pd.Series(np.where(test_df.Loan_Status.values == 'Y', 1, 0),
          test_df.index)

In [4]:
test_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
X.fillna(X.mean(),inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [8]:

expected_y  = y_test
y_pred = model.predict(X_test)
print(metrics.classification_report(expected_y, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.51      0.62        49
           1       0.80      0.93      0.86       105

    accuracy                           0.80       154
   macro avg       0.79      0.72      0.74       154
weighted avg       0.80      0.80      0.79       154



In [9]:
from sqlalchemy import create_engine

In [10]:
rds_connection_string = "postgres:postgres@localhost:5432/loan_data"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
test_df.to_sql(name='data', con=engine, if_exists='append', index=True)